In [1]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

In [2]:
# webscraping function used to grab team salary data from 1990 - 2021 from https://hoopshype.com/salaries/2019-2020/
# The data presented on this website was from the HoopsHype's salary database and the adjusted for inflation data points
# were provided by Current US Dollars adjusted for inflation from data provided by the U.S.
# Department of Labor Bureau of Labor Statistic
# The function is based of the code from Syed Sadat Nazrul written for a tutorial on Web Scraping with python
# written on Jul 25,2018 https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
def salary(url,year):
    url=url
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    col = []
    i=0

    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        print ('%d:"%s"'%(i,name))
        col.append((name,[]))
    for j in range(1,len(tr_elements)):
        T=tr_elements[j]
        if len(T)!=4:
            break
        i=0
        for t in T.iterchildren():
            data=t.text_content()
            #check if row is empty
            if i>0:
                try:
                    data=int(data)
                except:
                    pass
            col[i][1].append(data)
            i+=1

    [len(c) for (title,c) in col]

    dict = {title:column for (title,column)in col}
    df=pd.DataFrame(dict)

    team = df['Team'].str.strip('\n\t')
    rank = df[''].str.strip('\n\t.')
    salary = df.iloc[:,2].str.strip('\n\t$')
    salary2 = df.iloc[:,3].str.strip('\n\t$')
    year = [year]*27

    teamSalary = pd.DataFrame({
        "rank":rank[0:27],
        "team":team[0:27],
        "salary": salary[0:27],
        "salary(Inflation)":salary2[0:27],
        "Year":year
    })
    return teamSalary

#df = salary("https://hoopshype.com/salaries/2020-2021/",2021)
#df.to_csv(r'D:\UofC2021\DATA601\Project\Data601_Project\salary21.csv',index = False)

# Exploration of NBA Salaries
## Introduction

The focus of our exploratory data analysis will be on the salaries of athletes in the National Basketball Association (NBA).
In exploring this data, we hope to examine how total salaries are divided amongst the teams and the allocation amongst
players within certain teams. The earnings of top athletes are frequently addressed in the media and our team would like
to explore the magnitude of the pay gap between top-performing teams and the best individual athletes relative to others.
The salaries of athletes have drawn attention due to the announcement of large contracts for star players. Our team will
provide an illustration of how these salaries have changed over the past [x years]. The growth rate in these metrics will
be compared to wage growth data of the general public to see if there is significant divergence.

## Guiding Questions

1. How are the salaries of NBA players divided amongst the teams in the league and amongst the players within these teams?

    In exploring this question, we would like to look at the allocation of the league overall and examine some constituent
    teams, some containing star power and others which do not. In doing so, we would like to identify how the allocations
    differ when there are star players on a team.


2. Is there a significant correlation between the salaries paid and the success of a team?

    This question would assist in determining whether higher pay leads to better performance and if the ability for a
    team to succeed is dependent on the salaries paid.


3. How are player salaries related to the position played? How have these changed over time?

    Exploring these questions will identify if certain positions are more highly valued and provide insight into how
    team compositions may change over time.


4. How rapidly have salaries of NBA athletes increased over the past [x years] and how does this compare with average
wage growth in the United States over the same period?

    The contracts for certain athletes seem to grow faster than average wages. By studying the growth rate of the
    athletes of the NBA compared to national labor statistics, we would like to see if there is a divergence.

## Dataset

ESPN publishes NBA player salaries from 1990 to 2021. This page is organized in a structured table with headings for
Rank, Name, Position, Team and, Salary (USD). We will build and use a web scraper to pull this data into same columns and
export it into a comma separated file (CSV). ESPN is part of The Walt Disney Company.

For the NBA team salaries, we will get the data from the website HoopsHype which is a website from USA Today Sports.
HoopsHype has team salary data 1990 to 2021 which is organized and displayed in a structured html table with columns:
Rank, Team, Salary (USD), and Salary adjusted for inflation (USD) using the U.S. Department of Labor Bureau of Labor Statistic.
Again, we will be scraping by building a web scraper and export each year into CSV files. We will be stitching the
multiple CSV files into a single one and wrangle the data usable for data analysis.

For any additional player and game information we will be using the website basketball-reference.com. The information
provided on this website is organized into structured tables which can be scraped if needed. Player data is separated
into Regular season and a Playoff table with 30 columns that include: Season, Age, Team (Tm), League (Lg), Position (Pos),
Games (G), Games Started (GS), Minutes played per game (MP), Field Goals per game (FG), Field Goal attempts per game (FGA),
Field Goal percentage (FG%), 3-Points Field Goals per game (3P), 3-Point Field Goal attempts per game (3PA),
3-Point Field Goal percentage (3P%), 2-Point Field Goals per game (2P), 2-Point Field Goal attempts per game (2PA),
2-Point Field Goal percentage (2P%), Effective Field Goal percentage (eFG%), Free Throws per game (FT),
Free Throw Attempts per game (FTA), Free Throw Percentage (FT%), Offensive Rebounds per game (ORB), Defensive Rounds per game (DRB),
Total Rounds per game (TRB), Assists per game (AST), Steals per game (STL), Blocks per game (BLK), Turnovers  per game (TOV),
Personal Fouls per game (PF), Points per game (PTS). These tables are available to downloaded in a CSV format.
Basketball-reference is owned and operated by Sports Reference LLC.

## Tasks

### Packages
Pandas, Matplotlib, NumPy, requests, lxml, seaborn

#### Web scraping
For our web scraping function, we used the package *requests* to make HTTP requests to retrieve data from a specific URL.
We will also need to make use of *lxml* package, to handle XML and HTML tables. *Pandas* was used to convert the html
tables to a dataframe which makes easier for data wrangling.

#### Data Wrangling
Our data still needs to be cleaned and organized to be able to do more Exploratory Data Analysis (EDA) on our data.
For this, we will use *Pandas* and *Numpy* packages. Tables retrieved from HoopsHype have extra characters in the table
that need to be removed (\n\t,$). We will also need to stitch together multiple CSV files to a master CSV file and organize
the dataframe by teams. For some of our visuals, we will need to filter our data based on players position which will completed
using *Pandas* package. *Numpy* will be used for any additional calculations that and *Pandas* will be used to add those
values to dataframe.

#### Visualization
In order to create informative visuals, we will be applying different filters to our cleaned data. As our data consists
of both individual player and team information, we will need to appropriately select subjects of interest. Time series
data will likely be illustrated using a line graph but, we may consider other types if further insight may be gained.
To show the allocation of salaries among teams in the league, a pie chart would provide a clear visual of this distribution.
A bar of pie chart allows for more detailed inspection of specific teams from the overall set.

We may also do some basic calculations in order to develop some visual comparisons. In examining the salaries based on
position, we will calculate the mean/median for each player in the position during each season in order to develop a
broader understanding of how the position played impacts the salary received. A word cloud or heat map may allow for
visualization of the relative magnitudes based on position [note: word cloud would need additional libraries beyond pandas, numpy, matplotlib]
Following our exploration of the change in wages for both players in general as well as per position since 1991(?),
we will hone in our focus on the wage discrepancy within teams currently. Three different datasets will be evaluated,
each one representing a specific NBA team's salary sheet in the 2021-2022 season. Each team reflects a different
perspective and philosophy of wage distribution in the league, which includes a team with one superstar, a well-rounded
team with no dominant player and a team consisting of a mix of young players and role-players. We will calculate the
mean, median, standard deviation as well as the percent of the total salary each player takes up to illuminate the
distribution of wealth for each situation. Furthermore, a pie-chart will be utilized to visualize the data for each team,
and each of these charts will be put side by side for comparison.